In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



## Load dataset

In [4]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = ut.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')


train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 300, 300, 300)




train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



In [3]:
#check distribution of classes 

print("Check overlap between datasets:")
print("Train Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("Train Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("Validation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print("Train Subset Distribution:")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:
Train Subset and Validation Subset:
No actual data overlap detected.
Train Subset and Test Subset:
No actual data overlap detected.
Validation Subset and Test Subset:
No actual data overlap detected.
Train Subset Distribution:
Total samples in subset: 296
Model ADM, Class ai: 20 (6.76%)
Model ADM, Class nature: 17 (5.74%)
Model BigGAN, Class nature: 20 (6.76%)
Model BigGAN, Class ai: 17 (5.74%)
Model Midjourney, Class ai: 21 (7.09%)
Model Midjourney, Class nature: 16 (5.41%)
Model VQDM, Class nature: 11 (3.72%)
Model VQDM, Class ai: 26 (8.78%)
Model glide, Class nature: 18 (6.08%)
Model glide, Class ai: 19 (6.42%)
Model stable_diffusion_v_1_4, Class nature: 22 (7.43%)
Model stable_diffusion_v_1_4, Class ai: 15 (5.07%)
Model stable_diffusion_v_1_5, Class nature: 19 (6.42%)
Model stable_diffusion_v_1_5, Class ai: 18 (6.08%)
Model wukong, Class ai: 22 (7.43%)
Model wukong, Class nature: 15 (5.07%)

Validation Subset Distribution:
Total samples in subset: 29

In [9]:


class HighPassFilters(nn.Module):
    def __init__(self, kernels):
        super(HighPassFilters, self).__init__()
        # Kernels are a parameter but not trained
        self.kernels = nn.Parameter(kernels, requires_grad=False)

    def forward(self, x):
        # Apply convolution with padding to maintain output size equal to input size
        return F.conv2d(x, self.kernels, padding =2)  # Padding set to 2 to maintain output size



## CNN

In [8]:

class CNNBlock(nn.Module):
   def __init__(self, kernals):
       super(CNNBlock, self).__init__()
       self.conv = nn.Conv2d(30, 3, kernel_size=1,padding=0)
       self.filters = HighPassFilters(kernals)
       self.bn = nn.BatchNorm2d(3)
       self.htanh = nn.Hardtanh()
   def forward(self, x):
       x = self.filters(x)
       x = self.conv(x)
       x = self.bn(x)
       x = self.htanh(x)
       return x
  

## Model

In [6]:
class ImageClassificationModel(nn.Module):
    def __init__(self,kernels):
        super(ImageClassificationModel, self).__init__()
        self.feature_combiner = CNNBlock(kernels)
        self.feature_combiner2 = CNNBlock(kernels)
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits


## Train & Validation

In [2]:

kernels = ut.apply_high_pass_filter()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageClassificationModel(kernels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW([
    {'params': model.feature_combiner.parameters(), 'lr': 1e-5,},
    {'params': model.feature_combiner2.parameters(), 'lr': 1e-5,},
    {'params': model.transformer.parameters(), 'lr': 1e-5,}
])
# #freeze the transformer
# for param in model.transformer.parameters():
#     param.requires_grad = False
# #unfreeze classifier
# for param in model.transformer.classifier.parameters():
#     param.requires_grad = True
    
best_val_accuracy = 0.0

#
# Try to load previous best model and its best validation accuracy


def train_and_validate(model, train_loader, valid_loader, optimizer, 
                       device, num_epochs,best_model_path):
    
    try:
        checkpoint = torch.load(best_model_path)
        best_val_accuracy = checkpoint['best_val_accuracy']
        print("Loaded previous best model with accuracy:", best_val_accuracy)
    except FileNotFoundError:
        best_val_accuracy = float('-inf')
        print("No saved model found. Starting fresh!")

    for epoch in range(num_epochs):
        # # Training Phase
        model.train()
        total_train_loss, total_train, correct_train = 0, 0, 0
        for batch in train_loader:
            rich, poor, labels, model_names = batch  # Unpack model_names as well
            rich = rich.to(device)
            poor = poor.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(rich, poor)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = total_train_loss / total_train
        train_accuracy = correct_train / total_train

        # Validation Phase
        model.eval()
        total_val_loss, total_val, correct_val = 0, 0, 0
        with torch.no_grad():
            for batch in valid_loader:
                rich, poor, labels, model_names = batch  # Unpack model_names as well
                rich = rich.to(device)
                poor = poor.to(device)
                labels = labels.to(device)

                outputs = model(rich, poor)
                loss = criterion(outputs, labels)

                total_val_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = total_val_loss / total_val
        val_accuracy = correct_val / total_val

        # Print overall validation accuracy
        print(f'Epoch {epoch+1}/{num_epochs}\n,'
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}\n')
        
        # Check if general accuracy is the best and save
        if val_accuracy > best_val_accuracy:
            best_val_accuracy_general = val_accuracy
            torch.save({'model_state': model.state_dict(),
                        'best_val_accuracy': best_val_accuracy_general},
                       best_model_path)
            print(f"Saved new best model with accuracy: {best_val_accuracy_general:.4f}")
best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_newPatching_fixed_subset.pth'
train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path=best_model_path)


NameError: name 'ut' is not defined

## Test

In [12]:
kernels = ut.apply_high_pass_filter()   
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageClassificationModel(kernels).to(device)

def test(model, test_loader, device):
    # Load the best model
    checkpoint = torch.load("/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_unfrozen.pth")
    model.load_state_dict(checkpoint['model_state'])
    
    model.eval()
    total_test, correct_test = 0, 0
    test_accuracy_per_model = defaultdict(lambda: {'correct': 0, 'total': 0})

    with torch.no_grad():
        for batch in test_loader:
            rich, poor, labels, model_names = batch  # Assuming you have model_names
            rich = rich.to(device)
            poor = poor.to(device)
            labels = labels.to(device)

            outputs = model(rich, poor)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            # Collect stats per model just like validation phase
            for model_name, pred, true in zip(model_names, predicted, labels):
                test_accuracy_per_model[model_name]['total'] += 1
                if pred == true:
                    test_accuracy_per_model[model_name]['correct'] += 1

    test_accuracy = correct_test / total_test
    print(f'Test Accuracy: {test_accuracy:.4f}')

    # Print per model accuracy
    print("-------------------------------------------------------------------------")
    print("Test Accuracy per model:")
    for model_name, stats in test_accuracy_per_model.items():
        model_accuracy = stats['correct'] / stats['total']
        print(f"Test Accuracy for model {model_name}: {model_accuracy:.4f}")

ut.test(model, test_loader, device,"/home/kosta/code/School/SentryAI/pth/best_model_newPre_SWIN_unfrozen.pth")


You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


FileNotFoundError: [Errno 2] No such file or directory: '/home/kosta/code/School/SentryAI/pth/best_model_newPatching.pth'